In [1]:
from tabula import read_pdf
import pandas as pd
from openpyxl import load_workbook
from PyPDF2 import PdfReader, PdfWriter
from datetime import datetime
import ipywidgets as widgets
import io
from IPython.display import display
import getpass

In [2]:
!jupyter nbextension enable --py widgetsnbextension --sys-prefix
!jupyter serverextension enable voila --sys-prefix

Enabling notebook extension jupyter-js-widgets/extension...
      - Validating: ok
Enabling: voila
- Writing config: C:\Users\Pichau\anaconda3\envs\Mae\etc\jupyter
    - Validating...
      voila 0.3.6 ok


In [3]:
text_1 = widgets.HTML(value="<h1>Pedido:</h1>") 
file_upload = widgets.FileUpload(
    accept='*.pdf',  
    multiple=False
)
completion_text = widgets.Label(
    value='Magica Feita!!',
)
completion_text.layout.visibility = 'hidden'
button = widgets.Button(
    description='Magica',
    disabled=False,
    button_style='success', # 'success', 'info', 'warning', 'danger' or ''
    tooltip='Magica',
    icon='check'
)
interface = widgets.VBox([file_upload, button, completion_text])
interface

In [4]:
def on_button_clicked(event):
#Lendo o arquivo e transformando em BytesIO para ser interpretado
    keys = [*file_upload.value]
    filename = str(keys[0])
    content = file_upload.value
    content = content[filename]['content']
    content = io.BytesIO(content)
    
#Pegando o texto do arquivo
    reader = PdfReader(content)
    page = reader.pages[0]
    text = page.extract_text()

#cortando o arquivo
    page.cropbox.upper_right = (842, 380)

#salvando o arquivo cortado
    writer = PdfWriter()
    writer.addPage(page)
    writer.write("a.pdf")

#Tranformando o arquivo cortado em Dataframe
    table = read_pdf("a.pdf", pages="1")
    table = table[0]

#Tratando a DF
    new_row = table.columns
    table.loc[len(table.index)] = new_row
    table.columns = ['a', 'b', 'Cod', 'd', 'a', 'd', 'Quant', 'f', 'g', 'h', 'i', 'j', 'k', 'l', 'm']
    table = table[['Cod', 'Quant']]

#Lendo o gabarito, que é o molde do arquivo final
    gabarito_wb = load_workbook('GABARITO_jul22.xlsx')
    gabarito = gabarito_wb['Gabarito']
    
#Lendo tabela com os preços
    df = pd.read_excel('Offpaper_Atacado.xlsx')
    preco_tbl = df[['Referencia', 'PREÇO']]
    code = table["Cod"].tolist()
    code = [int(item) for item in code]
    cont = 0
    lista_preco = []
    lista_ref = []
    
#Atribuindo os preços para cara código na ordem certa
    for cod in code:
        for value in preco_tbl['Referencia']:
            if int(value) == int(cod):
                index = preco_tbl['Referencia'].tolist().index(value)
                preco = preco_tbl['PREÇO'][index]
                lista_preco.append(preco)
                lista_ref.append(value)

    Quantidade = table["Quant"].tolist()
    cont = 0

#Colocando na tabela a quantidade e preço de acordo com os códigos, ref é código
    for ref in lista_ref:
        for celula in gabarito['B']:
            try:
                cell = celula.value.split(" - ")
                if cell[0].isnumeric():
                    if int(ref) == int(cell[0]):
                        linha = celula.row
                        gabarito[f'E{linha}'] = Quantidade[cont]
                        gabarito[f'F{linha}'] = lista_preco[cont]
                        cont += 1
                        break
            except:
                pass
            

#tratando o text do PDF para obter as informações necessárias
    def cropr(doc, start, end):
        return doc[doc.find(start):doc.rfind(end)]

    text = cropr(text, "Endereço", "Bairro")
    text = cropr(text, "Endereço", "Bairro")
    text = cropr(text, "Endereço", "Bairro")
    text = text.split(": ")

    cnpj = text[4].split(":")
    cnpj = cnpj[2].split("\n")
    
    insc = cnpj[1]
    cnpj = cnpj[0]



    endereco = text[5].split("\n")
    bairro = endereco[1].split(" ")
    cep = endereco[2].split(" ")
    bairro = bairro[0]
    cep = cep[0]

    endereco = endereco[0]
    endereco = cropr(endereco, "RUA", "RUA")
    endereco = cropr(endereco, "RUA", "RUA")

    cidade = text[0]
    cidade = cidade.split("\n")
    cidade = cidade[1]
    cidade = cidade[0:cidade.rfind("Cidade")].split(" ")
    cidade = list(filter(None, cidade))
    estado = cidade[len(cidade)-1]
    cidade.pop()
    cidade = ' '.join(cidade)

    gabarito['C7'].value = 'BIGNARDI IND.COM.DE PAP.E ARTEF.LTDA'
    gabarito['F7'].value = cnpj
    gabarito['J7'].value = insc
    gabarito['C8'].value = endereco
    gabarito['E8'].value = bairro
    gabarito['I8'].value = cidade
    gabarito['M8'].value = estado
    # gabarito['C9'].value = 'LUCAS SANTOS'
    gabarito['C6'].value = 'Penpel Representações'
    gabarito['J6'].value = datetime.today().strftime('%d/%m/%Y')
#Salvando Tabela e exibindo texto de finalização
    completion_text.layout.visibility = 'visible'
    gabarito_wb.save('C:/Users/'+getpass.getuser()+'/Downloads/Pedro.xlsx')
    
button.on_click(on_button_clicked)

In [6]:
!jupyter nbconvert --to script Outro.ipynb


[NbConvertApp] Converting notebook Outro.ipynb to script
[NbConvertApp] Writing 4883 bytes to Outro.py
